In [1]:
using JuMP
using DataFrames
using Gurobi
using CSV

In [8]:
m = Model(solver=GurobiSolver(MIPGap=.1))

#Read in the data
data = CSV.read("supermarket_data.csv")
simple_dist = CSV.read("distmatrix_simple.csv")

#Puts the data into an array
v = data[:,2]

#Puts distance into an array
simple_dist = simple_dist[:,2:103]
c = simple_dist
    
@variables m begin
    y[1:102], Bin # 1 if item i is taken
end

@variables m begin
    x[1:102,1:102], Bin # 1 if direct path betweeen i and j
end

@variables m begin
    z[1:102] #total time to j if item j is picked up
end

@variables m begin
    t[1:102,1:102]  #total time to j if path i,j is in the optimal solution
end

@constraint(m, y[1] == 1)  #We start at y[1]
@constraint(m, y[102] == 1)  #We must end at the start node
@constraint(m, z[1] == 0)    #time from the first node is 0
@constraint(m, sum(y[i] for i in 2:101) <= 15)  #We can take at most 15 items
@constraint(m, sum(x[i,1] for i in 1:102) == 0)  #The start node is the beginning path
@constraint(m, sum(x[102,j] for j in 1:102) == 0)  #The start node is the ending path

for i in 1:101
    @constraint(m, sum(x[i,j] for j in 2:102) == y[i]) # For each node that is included in the path (except for the end node), there is another node that directly follows it.
end

for j in 2:102
    @constraint(m, sum(x[i,j] for i in 1:101) == y[j])  #For each node that is included in the path (except for the start node), there is another node that directly precedes it
end

for j in 2:102
    @constraint(m, sum(t[i,j] for i in 1:101) == z[j])  #Establishes  the  relationship  between tij and zj,  as discussed in our description of decision variables for all nodes except for the start node.
end

for j in 1:101
    @constraint(m, sum(t[j,k] for k in 2:102) == (z[j] + sum(c[j,k]x[j,k] for k in 2:102)))  #The total distance from the start node to another node (k) on the path is equal to the distance to the preceding node (j) inthe path plus the distance from j to the next node (k), for all nodes j except for the end node.
end


# If the direct path between two nodes is taken,the total 
#distance from the start node to the second node in this 
#direct path is at most 90.  In other words, the entire path can take a maximum of 90 seconds.
for j in 1:102
    for k in 2:102
        @constraint(m, t[j,k] >= x[j,k])
        @constraint(m, t[j,k] <= 90x[j,k])
    end
end

@objective(m, Max, sum(v[i]*y[i] for i in 1:102))

solve(m)

Academic license - for non-commercial use only
Optimize a model with 21014 rows, 21012 columns and 92923 nonzeros
Variable types: 10506 continuous, 10506 integer (10506 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [9e-01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 16.1700000
Presolve removed 711 rows and 749 columns
Presolve time: 0.55s
Presolved: 20303 rows, 20263 columns, 90689 nonzeros
Variable types: 10000 continuous, 10263 integer (10263 binary)

Root relaxation: objective 1.629900e+02, 2095 iterations, 0.37 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  162.94956    0   36   16.17000  162.94956   908%     -    6s
H    0     0                     102.9100000  162.94956  58.3%     -    6s
H    0     0                     157.2900000  162.94956  3.60% 

:Optimal

In [24]:
println("Items Taken (In Order):")
cnode = 1
while cnode != 102
    for i in 1:102
        if getvalue(x[cnode,i]) == 1
            cnode = i
            if cnode != 102
                println(data[cnode,1], "(Item #: ",  cnode - 1, ", ", "Price: ", data[cnode,2] , ")")
            end
            break
        end
    end
end

println("")

println("Objective Value: ", getobjectivevalue(m))

println("Total Time: ", getvalue(z[102]))

Items Taken (In Order):
K-Cups(Item #: 2, Price: 10.99)
Coffee Beans(Item #: 1, Price: 6.99)
Diapers(Item #: 33, Price: 25.99)
Advil(Item #: 36, Price: 6.29)
Shampoo(Item #: 40, Price: 8.99)
Steak(Item #: 91, Price: 7.13)
Bacon(Item #: 92, Price: 7.99)
Toilet Paper(Item #: 41, Price: 7.99)
Roses (12)(Item #: 100, Price: 12.99)
Pizza (12")(Item #: 84, Price: 8.99)
Lasagna(Item #: 82, Price: 9.99)
Taquitos(Item #: 79, Price: 6.99)
Broom(Item #: 51, Price: 13.99)
Detergent(Item #: 49, Price: 12.99)
Trash Bags(Item #: 47, Price: 8.99)

Objective Value: 157.29
Total Time: 90.0
